In [1]:
import pandas as pd

# Load the dataset with only the necessary columns
selected_columns = ['Severity', 'Description', 'Street', 'City', 'State', 
                    'Country', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
                    'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 
                    'Wind_Speed(mph)', 'Weather_Condition']

df = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_US_accidents/archive/US_Accidents_March23.csv', usecols=selected_columns)

# Display the first few rows to check the data
print(df.head())


   Severity                                        Description  \
0         3  Right lane blocked due to accident on I-70 Eas...   
1         2  Accident on Brice Rd at Tussing Rd. Expect del...   
2         2  Accident on OH-32 State Route 32 Westbound at ...   
3         3  Accident on I-75 Southbound at Exits 52 52B US...   
4         2  Accident on McEwen Rd at OH-725 Miamisburg Cen...   

                      Street          City State Country  Temperature(F)  \
0                     I-70 E        Dayton    OH      US            36.9   
1                   Brice Rd  Reynoldsburg    OH      US            37.9   
2             State Route 32  Williamsburg    OH      US            36.0   
3                     I-75 S        Dayton    OH      US            35.1   
4  Miamisburg Centerville Rd        Dayton    OH      US            36.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi) Wind_Direction  \
0            NaN         91.0         29.68            10.0         

In [2]:
# Handle missing values - example with 'Wind_Speed(mph)'
df['Wind_Speed(mph)'].fillna(df['Wind_Speed(mph)'].mean(), inplace=True)

# Frequency encoding for categorical columns
categorical_columns = ['Description', 'Street', 'City', 'State', 'Country', 'Weather_Condition', 'Wind_Direction']

for col in categorical_columns:
    freq = df[col].value_counts()
    df[col] = df[col].map(freq)

# Drop any remaining rows with missing values if necessary
df = df.dropna()

# Display the first few rows after preprocessing
print(df.head())


   Severity  Description   Street     City   State  Country  Temperature(F)  \
2         2          1.0    252.0   3010.0  118115  7728394            36.0   
3         3          5.0  27546.0  24572.0  118115  7728394            35.1   
4         2          2.0    729.0  24572.0  118115  7728394            36.0   
5         3          1.0    104.0   1073.0  118115  7728394            37.9   
6         2          1.0     26.0  24572.0  118115  7728394            34.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi)  Wind_Direction  \
2           33.3        100.0         29.67            10.0        364470.0   
3           31.0         96.0         29.64             9.0        364470.0   
4           33.3         89.0         29.65             6.0        364470.0   
5           35.5         97.0         29.63             7.0        384840.0   
6           31.0        100.0         29.66             7.0        353806.0   

   Wind_Speed(mph)  Weather_Condition  
2         

In [3]:
# Define features (X) and target (y)
X = df.drop(columns=['Severity'])
y = df['Severity']

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (5676313, 13), y shape: (5676313,)


In [4]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil
import pandas as pd

# Function to calculate and return metrics
def calculate_metrics(X_train, X_test, y_train, y_test):
    gbr = GradientBoostingRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [50],  # Number of boosting stages
        'learning_rate': [0.1],  # Learning rate
        'max_depth': [3],  # Depth of individual estimators
        'min_samples_split': [5],  # Min samples to split
        'min_samples_leaf': [2],  # Min samples in leaf
        'subsample': [0.8]  # Subsampling for training
    }

    random_search = RandomizedSearchCV(gbr, param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,
        'Execution Time (Raw)': execution_time,
        'Normalized Time (s/MB)': normalized_time,
        'Average CPU Usage': avg_cpu_usage
    }


In [8]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Metrics for sample size 1.0:
RMSE: 0.43379474744429514
MAPE: 0.1163801425995731
R2: 0.11489929118557474
MSE: 0.1881778829102598
nRMSE: 0.14459824914809838
Execution Time (Raw): 1325.296961069107
Normalized Time (s/MB): 2.1880806325434343
Average CPU Usage: 44.75
Sample Size: 1.0
--------------------------------------------------


c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Metrics for sample size 0.5:
RMSE: 0.4299059369914377
MAPE: 0.1125393379416248
R2: 0.11295131299556938
MSE: 0.184819114660486
nRMSE: 0.1433019789971459
Execution Time (Raw): 620.1266582012177
Normalized Time (s/MB): 2.0456250301692367
Average CPU Usage: 11.8
Sample Size: 0.5
--------------------------------------------------


c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


...

Metrics for sample size 1000:
RMSE: 0.45966361534625805
MAPE: 0.10348688815196287
R2: 0.052402111119216555
MSE: 0.21129063927319267
nRMSE: 0.15322120511541934
Execution Time (Raw): 2.1668999195098877
Normalized Time (s/MB): 25.358920200892857
Average CPU Usage: 10.8
Sample Size: 1000
--------------------------------------------------


c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Metrics for sample size 10000:
RMSE: 0.43421427762704934
MAPE: 0.12188268658383729
R2: 0.12067896013655444
MSE: 0.18854203889518026
nRMSE: 0.1447380925423498
Execution Time (Raw): 3.616272211074829
Normalized Time (s/MB): 4.232071707589285
Average CPU Usage: 11.95
Sample Size: 10000
--------------------------------------------------


c:\Users\manpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Metrics for sample size 100000:
RMSE: 0.42733441604496053
MAPE: 0.11325189886399048
R2: 0.10763862681534775
MSE: 0.18261470313648742
nRMSE: 0.14244480534832019
Execution Time (Raw): 12.160685062408447
Normalized Time (s/MB): 1.4231476004464285
Average CPU Usage: 9.5
Sample Size: 100000
--------------------------------------------------


In [9]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 42 minutes and 3.02 seconds
Total Normalized Execution Time for Entire Process: 2.20218138 seconds per MB
Total Average CPU Usage for Entire Process: 14.88%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.433795,0.116380,0.114899,0.188178,0.144598,1325.296961,2.188081,44.75,1.000
1,0.429906,0.112539,0.112951,0.184819,0.143302,620.126658,2.045625,11.80,0.500
2,0.430146,0.112410,0.111959,0.185026,0.143382,424.109939,2.798041,10.55,0.250
3,0.430004,0.112380,0.112548,0.184903,0.143335,133.630721,1.763242,9.40,0.125
4,0.394920,0.111770,0.400148,0.155962,0.131640,1.911847,223.740737,10.25,100.000
5,0.459664,0.103487,0.052402,0.211291,0.153221,2.166900,25.358920,10.80,1000.000
6,0.434214,0.121883,0.120679,0.188542,0.144738,3.616272,4.232072,11.95,10000.000
7,0.427334,0.113252,0.107639,0.182615,0.142445,12.160685,1.423148,9.50,100000.000
